In [ ]:
# packages
#pip install neo4j-driver
#pip install openpyxl
#pip install itertools

In [ ]:
# imports
from neo4j import GraphDatabase, basic_auth
import pandas as pd
from openpyxl import Workbook, load_workbook
from itertools import islice

In [ ]:
# set driver connection
driver = GraphDatabase.driver(
  "neo4j://54.174.149.140",
  auth=basic_auth("neo4j", "portions-prop-mosses"))

In [26]:
# define query functions
def name_query(name):
    cypher_query = f'''
    MATCH (b:Officer)
    WHERE toLower(b.name) = toLower("{name}") 
    RETURN *
    '''
    with driver.session(database="neo4j") as session:
        results = session.execute_read(lambda tx: tx.run(cypher_query, name=name).data())
        if len(results) > 0:    
            print(f'{name}: {len(results)} results')
        driver.close()


def city_query(city):
    cypher_query = f'''
    MATCH (b)
    WHERE toLower(b.name) CONTAINS toLower("{city}")
    RETURN  *
    '''
    with driver.session(database="neo4j") as session:
        results = session.execute_read(lambda tx: tx.run(cypher_query).data())
        if len(results) > 0:
            print(f'{city}: {len(results)} results')
        driver.close()

## Niedersachsen

In [23]:
# import excel file (.xlsx)
book = load_workbook("Rechercheliste_Niedersachsen.xlsx")
sheet = book.active

data = sheet.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)

# convert worksheet to a Dataframe
df = pd.DataFrame(data, index=idx, columns=cols)
df.index.names = ['Name']
df.head()

,Geburtsdatum,Wohnort,Firmensitz,NaN,NaN,NaN
Name,,,,,,
Martin Kind,1944-04-28 00:00:00,Burgwedel,None,None,None,None
Kind GmbH & Co. KG,None,None,"Kokenhorststraße 3-5, 30938 Großburgwedel",None,None,None
Dirk Roßmann,1976-09-07 00:00:00,"Wittenbecksweg 20, 29646 Bispingen",None,None,None,None
Raoul Roßmann,1986-08-10 00:00:00,"Blücherstr. 5, 30175 Hannover",None,None,None,None
Dirk Daniel Roßmann,1976-07-23 00:00:00,"Riethof 6 B, 30916 Isernhagen",None,None,None,None


In [24]:
# Personen
allPersons = list(df.index)
for name in allPersons:
    name_query(name)

Carsten Maschmeyer: 1 results


In [27]:
# Wohnort
wohnort = list(df.Wohnort)
cities = []
for x in wohnort:
    if x is not None and cities.count(x) == 0:
        cities.append(x)

for city in cities:
    city_query(city)

Burgwedel: 3 results
Hannover: 53 results
Leer: 154 results
Dissen: 2 results
Burgdorf: 4 results
Papenburg: 1 results
Ludwigsburg: 1 results
Braunschweig: 8 results
Visbek: 4 results
Vaduz: 215 results
München: 3 results
Duderstadt: 1 results


In [28]:
# Adresse
adresse = list(df.Firmensitz)
adress_list = []
for x in adresse:
    if x is not None and adress_list.count(x) == 0:
        adress_list.append(x)

for adress in adress_list:
    city_query(adress)

Langenhagen: 1 results
Luxemburg: 16 results
Am Weinberg 2, 37115 Duderstadt: 1 results


## Hamburg

In [29]:
# import excel file (.xlsx)
book = load_workbook("Namensliste-Hamburg.xlsx")
sheet = book.active

data = sheet.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)

# convert worksheet to a Dataframe
df = pd.DataFrame(data, index=idx, columns=cols)
#df.index.names = ['Name']
df.head()

,Name,TO DO,DONE
NaN,None,None,None
NaN,None,None,None
NaN,None,None,None
Warburg Bank,Christian Olearius,None,None
NaN,Max Warburg,None,None


In [30]:
# Personen/Namen
allPersons = list(df.Name)

allPersons_list = []
for x in allPersons:
    if x is not None and allPersons_list.count(x) == 0:
        allPersons_list.append(x)

for name in allPersons_list:
    name_query(name)

Bertram Rickmers: 1 results


## Bremerhaven

In [31]:
# import excel file (.xlsx)
book = load_workbook("Bremerhaven.xlsx")
sheet = book.active

data = sheet.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)

# convert worksheet to a Dataframe
df = pd.DataFrame(data, index=idx, columns=cols)
df.index.names = ['Name']
df.head()

,Wohnort,Firma,Firmensitz,NaN,NaN,NaN
Name,,,,,,
Friedrich Dieckell,Bremerhaven,Friedrich Dieckell Verwaltungsgesellschaft mbH,"Grashoffstr. 7, D-27570 Bremerhaven",None,None,None
Dirk Grotheer,Osterholz-Scharmbeck,Dieckell Verwaltungs GmbH,"Grashoffstr. 7, 27570 Bremerhaven",None,None,None
Olga Wichmann,Schiffdorf,None,None,None,None,None
Tim Oliver Pohlenz,Geestland,None,None,None,None,None
NaN,None,None,None,None,None,None


In [32]:
# Personen
allPersons = list(df.index)

allPersons_list = []
for x in allPersons:
    if x is not None and allPersons_list.count(x) == 0:
        allPersons_list.append(x)

for name in allPersons_list:
    name_query(name)

In [33]:
# Wohnort
wohnort = list(df.Wohnort)
cities = []
for x in wohnort:
    if x is not None and cities.count(x) == 0:
        cities.append(x)

for city in cities:
    city_query(city)

Bremerhaven: 4 results
Osterholz-Scharmbeck: 1 results
Hamburg: 351 results
Berlin: 342 results
Bad Bederkesa: 1 results
Quickborn: 1 results
Cuxhaven: 1 results
Bremen: 66 results
Elmshorn: 2 results
Midlum: 1 results
Erlangen: 7 results


In [34]:
# Firma
company = list(df.Firma)
companies = []
for x in company:
    if x is not None and companies.count(x) == 0:
        companies.append(x)

for company in companies:
    city_query(company)

In [35]:
# Firmensitz
company = list(df.Firmensitz)
companies = []
for x in company:
    if x is not None and companies.count(x) == 0:
        companies.append(x)

for company in companies:
    city_query(company)